In [ ]:
#Problem - A
import sys
def findparent(p,x):
  if p[x]!=x:
    p[x]=findparent(p,p[x])
  return p[x]
def poweroffriendship(p,size,x,y):
  rootX=findparent(p,x)
  rootY=findparent(p,y)
  if rootX==rootY:
    return size[rootX]
  if size[rootX]<size[rootY]:
    p[rootX]=rootY
    size[rootY]+=size[rootX]
    return size[rootY]
  else:
    p[rootY]=rootX
    size[rootX]+=size[rootY]
    return size[rootX]
input=sys.stdin.readlines()
n,m=map(int,input[0].split())
p=[i for i in range(n+1)]
size=[1]*(n+1)
r=[]
for i in range(1,m+1):
  a,b=map(int,input[i].split())
  circle=poweroffriendship(p,size,a,b)
  r.append(circle)
print("\n".join(map(str,r)))

8 7
2 4
2
4 5
3
3 6
2
2 7
4
6 2
6
3 1
3
4 7
6


In [ ]:
#Problem - B
import sys
input=sys.stdin.readline
n,m=map(int, input().split())
l=[]
for _ in range(m):
  u,v,w=map(int, input().split())
  l.append((w,u,v))
l.sort()
p=list(range(n+1))
r=[0]*(n+1)
t=0
def find(p,x):
  if p[x]!=x:
    p[x]=find(p,p[x])
  return p[x]
def union(p,r,x,y):
  rtx=find(p,x)
  rty=find(p,y)
  if rtx!=rty:
    if r[rtx]<r[rty]:
      p[rtx]=rty
    elif r[rtx]>r[rty]:
      p[rty]=rtx
    else:
      p[rty]=rtx
      r[rtx]+=1
    return True
  return False
for w,u,v in l:
  if union(p,r,u,v):
    t+=w
print(t)

In [ ]:
#Problem - C
def find(parent, i):
  if parent[i] != i:
    parent[i] = find(parent, parent[i])
  return parent[i]
def union(parent, rank, x, y):
  xroot = find(parent, x)
  yroot = find(parent, y)
  if rank[xroot] < rank[yroot]:
    parent[xroot] = yroot
  elif rank[xroot] > rank[yroot]:
    parent[yroot] = xroot
  else:
    parent[yroot] = xroot
    rank[xroot] += 1
N, M = map(int, input().split())
edges = []
for _ in range(M):
  u, v, w = map(int, input().split())
  edges.append((u, v, w))
sorted_edges = sorted(edges, key=lambda x: x[2])
parent = [i for i in range(N + 1)]
rank = [0] * (N + 1)
mst = []
total_weight = 0
for edge in sorted_edges:
  u, v, w = edge
  if find(parent, u) != find(parent, v):
    union(parent, rank, u, v)
    mst.append(edge)
    total_weight += w
if len(mst) < N - 1:
  print(-1)
else:
  mst_adj = [[] for _ in range(N + 1)]
  for u, v, _ in mst:
    mst_adj[u].append(v)
    mst_adj[v].append(u)
  def get_component(start, forbid_u, forbid_v):
    visited = [False] * (N + 1)
    stack = [start]
    visited[start] = True
    component = {start}
    while stack:
      curr = stack.pop()
      for nei in mst_adj[curr]:
        if not visited[nei]:
          if not ((curr == forbid_u and nei == forbid_v) or (curr == forbid_v and nei == forbid_u)):
              visited[nei] = True
              component.add(nei)
              stack.append(nei)
    return component
  second_best = 10**18
  for e in mst:
    u, v, w_e = e
    C1 = get_component(u, u, v)
    C2 = get_component(v, u, v)
    min_f = 10**18
    for edge in edges:
      x, y, wf = edge
      if (x in C1 and y in C2) or (x in C2 and y in C1):
        if not ((x == u and y == v) or (x == v and y == u)):
          min_f = min(min_f, wf)
    if min_f < 10**18:
      candidate = total_weight - w_e + min_f
      if candidate > total_weight:
        second_best = min(second_best, candidate)
  if second_best == 10**18:
    print(-1)
  else:
    print(second_best)

In [ ]:
#Problem - C
import sys
input = sys.stdin.readline
n, m = map(int, input().split())
e = []
for _ in range(m):
  u, v, w = map(int, input().split())
  e.append((w, u, v))
p = list(range(n+1))
r = [0]*(n+1)
def find_parent(p, x):
  if p[x] != x:
    p[x] = find_parent(p, p[x])
  return p[x]
def union_mst(p, r, a, b):
  a = find_parent(p, a)
  b = find_parent(p, b)
  if a == b:
    return False
  if r[a] < r[b]:
    p[a] = b
  elif r[b] < r[a]:
    p[b] = a
  else:
    p[b] = a
    r[a] += 1
  return True
e.sort(key=lambda e: e[0])
mst_w = 0
mst_e = [[] for _ in range(n+1)]
used_e = [False]*m
for i, (w,u,v) in enumerate(e):
  if union_mst(p, r, u, v):
    mst_w += w
    mst_e[u].append((v,w))
    mst_e[v].append((u,w))
    used_e[i] = True
root = find_parent(p, 1)
if any(find_parent(p, i) != root for i in range(1, n+1)):
  print(-1)
  sys.exit()
log = (n+1).bit_length()
up = [[0]*(n+1) for _ in range(log)]
max_w = [[0]*(n+1) for _ in range(log)]
dp = [0]*(n+1)
def dfs(u, p):
  for v, w in mst_e[u]:
    if v == p:
      continue
    dp[v] = dp[u] + 1
    up[0][v] = u
    max_w[0][v] = w
    for k in range(1, log):
      up[k][v]    = up[k-1][ up[k-1][v] ]
      max_w[k][v] = max( max_w[k-1][v], max_w[k-1][ up[k-1][v] ] )
    dfs(v, u)
dfs(1, 0)
def query_max(u, v):
  if u == v:
    return 0
  if dp[u] < dp[v]:
    u, v = v, u
  diff = dp[u] - dp[v]
  ans = 0
  for k in range(log):
    if diff >> k & 1:
      ans = max(ans, max_w[k][u])
      u = up[k][u]
  if u == up[0][v]:
      return max_w[0][v]
  if v == up[0][u]:
      return max_w[0][u]
  for k in reversed(range(log)):
    if up[k][u] != up[k][v]:
      ans = max(ans, max_w[k][u], max_w[k][v])
      u = up[k][u]
      v = up[k][v]
  if u != v:
    ans = max(ans, max_w[0][u], max_w[0][v])
  return ans
second_best = float('inf')
for i, (w,u,v) in enumerate(e):
  if used_e[i]:
    continue
  mx = query_max(u, v)
  if mx < w:
    second_best = min(second_best, mst_w + w - mx)
print(second_best if second_best < float('inf') else -1)